In [79]:
import re
import pandas as pd
from requests import get
import nltk
from bs4 import BeautifulSoup 
import sys

tos_url = "http://chakoteya.net/StarTrek/episodes.htm"

response = get(tos_url)

if response.status_code >= 400:
    print("No good")
    sys.exit()

html = response.text

soup = BeautifulSoup(html)
soup

rows = list(soup.select("table tr"))

# Pop removes the first line so we can access the rest of the rows and season names
rows.pop(0).h1.text.split(" -")[0]

'The Star Trek Transcripts'

# Dataframe Format

- first pass: one show per row. `show, season, episode_name, production, airdate, transcript`
- second pass: break out each transcript into its own dataframe of dialogue, 

In [106]:
# show_name = response.url.split("/")[3]

episodes = []

for row in rows:
    output = {}
    
    if "Season" in str(row) or "Animated" in str(row):
        output["season"] = row.h2.text.strip()
        continue
    
    # This skips the header and also skips the unnecessary nested tbody included in this HTML
    if "Episode Name" in str(row):
        continue
    
    output["episode_name"] = row.select("td")[0].a.text.strip()
    output["link"] = row.select("td")[0].a["href"]
    output["production_order"] = int(row.select("td")[1].text)

    episodes.append(output)

In [107]:
df = pd.DataFrame(episodes)
df.episode_name = df.episode_name.str.replace(r"\s", " ", regex=True)

In [109]:
df["show_name"] = "Star Trek"
df["series_name"] = "The Original Series"

In [110]:
df

,episode_name,link,production_order,show_name,series_name
0,The Cage,1.htm,1,Star Trek,The Original Series
1,The Man Trap,6.htm,6,Star Trek,The Original Series
2,Charlie X,8.htm,8,Star Trek,The Original Series
3,Where No Man Has Gone Before,2.htm,2,Star Trek,The Original Series
4,The Naked Time,7.htm,7,Star Trek,The Original Series
...,...,...,...,...,...
97,Bem,TAS018.htm,22018,Star Trek,The Original Series
98,The Practical Joker,TAS021.htm,22021,Star Trek,The Original Series
99,Albatross,TAS019.htm,22019,Star Trek,The Original Series
100,How Sharper Than A Serpent's Tooth,TAS022.htm,22022,Star Trek,The Original Series
